# Cahier des charges

| Fonction | Cartographier le nombre de vélos disponibles dans un rayon de 500m |
| --- | --- |
| Objectif | Réaliser une couche raster du nombre de vélos disponibles dans un rayon de 500m à partir des données de l'API. |
| Contraintes | - Utiliser le package *rasterio*<br>- La couche doit couvrir toute zone située à moins de 5km d'une station Velo'V<br>- La résolution spatiale doit être de 50m |

## Résultat attendu
![Raster du nombre de vélos](img/nb_velos.png "Raster du nombre de vélos attendu")

# Algorithmie

Voici quelques grandes étapes pour vous aider à réaliser cet objectif. Ces étapes ne sont pas obligatoires, vous pouvez décider de procéder complètement autrement si vous le souhaitez.

> ⚠️ La méthode présentée ici n'est pas optimale en production, mais a été choisie pour sa simplicité algorithmique

## 1. Récupérer toutes les stations Vélo'V
Récupérer le tableau complet des stations Vélo'V et leurs attributs, notamment le nombre de vélos disponibles à chaque station.

## 2. Créer un raster aux bonnes dimensions et résolutions
On utilise le template_raster comme modèle pour en créer un nouveau.

## 3. Réaliser un masque des zones à plus de 5km d'une station
On réalise un buffer de 5km autour de chaque station (y compris celle qui n'ont pas de vélo disponible), puis on transforme ces buffers en un masque.

## 4. Calculer chaque pixel
Pour cette étape, on va réaliser une itération double : sur chaque ligne du raster, puis sur chaque colonne.

### 4.1. Chercher les stations présentes dans un rayon de 500m autour du pixel
On peux par exemple créer un point vectoriel au centre du pixel, faire un buffer autour de ce point puis une intersection spatiale avec les stations.

### 4.2. Faire la somme des vélos disponibles à ces stations
On fait la somme des vélos disponibles dans toutes les stations trouvées à l'étape précédente.

### 4.3. Assigner cette somme à la valeur du pixel
On stocke cette valeur dans le raster, à la position du pixel traité.

## 5. Ecrire le raster en sortie sur le disque
Une fois l'itération sur les pixels terminée, on peux enregistrer le raster sur le disque dur.

# Programmation

> ⚠️ La méthode présentée ici n'est pas optimale en production, mais a été choisie pour sa simplicité algorithmique

## 1. Récupérer toutes les stations Vélo'V
Refaire comme pour les objectifs précédents. Pas besoin de filtrage cette fois-ci.

## 2. Créer un raster aux bonnes dimensions et résolutions
Reprendre la méthode de l'objectif 3 (utiliser le template_raster).

## 3. Réaliser un masque des zones à plus de 5km d'une station
Reprendre la méthode de l'objectif 3 ou charger simplement le fichier vectoriel si vous l'avez enregistré sur le disque.

## 4. Calculer chaque pixel
Pour cette étape, on va réaliser une itération double : sur chaque ligne du raster, puis sur chaque colonne.

### 4.1. Chercher les stations présentes dans un rayon de 500m autour du pixel
Les objets de type ```shapely.Point``` ont une méthode qui permet de réaliser un buffer.
Pour l'intersection spatiale, la méthode ```GeoDataFrame.intersects(<shapely geometry>)``` renvoie une liste de booléens indiquant si chaque ligne du *GeoDataFrame* intersecte ou non la géométrie. Pour filtrer ensuite le *GeoDataFrame* en lui-même en utilisant cette liste, cherchez une solution dans la [pandas cheatsheet](https://pandas.pydata.org/Pandas_Cheat_Sheet.pdf).

### 4.2. Faire la somme des vélos disponibles à ces stations
On cherche à faire la somme de la colonne *num_bikes_available* du tableau filtré. La méthode pour faire cela avec un *pandas.DataFrame* est très simple à trouver.

### 4.3. Assigner cette somme à la valeur du pixel
Reprendre la méthode de l'objectif 3.

## 5. Ecrire le raster en sortie sur le disque
Reprendre la méthode de l'objectif 3.

# Autre algorithme plus optimisé

Plutôt que d'itérer sur les pixels, il est plus optimal d'itérer sur les stations et de mettre à jour tous les pixels autour de chaque station lors de chaque itération.

Toujours dans un soucis d'optimisation, on peux au préalable filtrer les stations qui n'ont aucun vélo disponible afin d'économiser quelques itérations.

Dans la boucle, il faudra utiliser rasterio.features.geometry_mask pour créer un masque raster aux bonnes dimensions autour de la station. Ensuite, on utilisera numpy pour faire de la "calculatrice raster" entre notre masque et le raster complet à mettre à jour.

In [ ]:
# On ouvre le template raster
with rio.open('datasets/output/template_raster.tif') as src:
    profile = src.profile
    data = src.read(1)

    # On itère sur chaque station avec plus d'un vélo
    for _, row in filtered_gdf.iterrows():

        # On fait un buffer de 500m autour de la station
        

        # On extrait le nombre de vélos dispos
        

        # On fait un "masque" du buffer
        
        
        # On l'inverse pour qu'il soit "True" dans le buffer
        

        # On multiplie le masque par le nombre de vélos avec numpy
        

        # On additionne ce tableau numpy au total
        

Utiliser plutôt cet algorithme pour le package pyvelov.